In [1]:
# import torch_geometric
import sys
import pickle
sys.path.insert(0, '../src')

In [39]:
import tqdm
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision
import json
import argparse
# import cv2
import numpy as np
import torch
from torch.autograd import Function
from torchvision import models
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import time
from torchvision.datasets import CocoDetection


from PIL import Image
import pandas as pd

sys.path.insert(0, '../src/data/cocoapi/PythonAPI')
from pycocotools.coco import COCO

from graph_network import GCNNet
from torch_geometric.data import DataLoader as TG_DataLoader

from dataset_class import *
from etl import *

In [3]:
from PIL import Image

In [4]:
### TODO: add target transform to convert labels to proper labels using id_dict

In [5]:
train_dir = '/datasets/COCO-2017/train2017'
val_dir = '/datasets/COCO-2017/val2017'

train_ann = '/datasets/COCO-2017/anno2017/instances_train2017.json'
val_ann = '/datasets/COCO-2017/anno2017/instances_val2017.json'

In [6]:
import pickle

In [7]:
# with open('../src/data/id_dict.pkl', 'wb') as f:
#     pickle.dump(id_dict, f)    

In [8]:
with open('../src/data/id_dict.pkl', 'rb') as f:
    id_dict = pickle.load(f)

In [9]:
def convert_ids(target):
    return id_dict[target]

In [10]:
# # %%capture
# train_coco, train_data = load_data(train_ann)
# %time drop_null_annotations(coco=train_coco, data=train_data, dataDir='/datasets/COCO-2017', dataType='train2017',\
#                             annFile=train_ann, overwrite=True, tmpDataDir='../data/temp');

In [11]:
# del train_coco
# del train_data

# del val_coco
# del val_data

In [12]:
clean_train_ann = '../data/temp/annotations/clean_instances_train2017.json'
clean_val_ann = '../data/temp/annotations/clean_instances_val2017.json'

In [13]:
%time train_set = cocoDataset(root=train_dir,\
                      annotation=clean_train_ann,\
                      transforms=get_transform(),\
                      target_transform=convert_ids)

%time val_set = cocoDataset(root=val_dir,\
                      annotation=clean_val_ann,\
                      transforms=get_transform(),\
                      target_transform=convert_ids)

loading annotations into memory...
Done (t=21.56s)
creating index...
index created!
CPU times: user 19.2 s, sys: 3.81 s, total: 23 s
Wall time: 23 s
loading annotations into memory...
Done (t=0.84s)
creating index...
index created!
CPU times: user 753 ms, sys: 136 ms, total: 890 ms
Wall time: 898 ms


In [14]:
# ## Params cell
num_epoch = 10
k = 3
fname = f'graph_80_{num_epoch}_{k}_resnet50'
train_batch_size = 12

In [15]:
trainloader = torch.utils.data.DataLoader(train_set,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=8,
                                          collate_fn=collate_fn)

valloader = torch.utils.data.DataLoader(val_set,
                                        batch_size=train_batch_size,
                                        shuffle=False,
                                        num_workers=8,
                                        collate_fn=collate_fn)

In [16]:
# Defining our model for GCN embeddings
model = models.resnet50(pretrained=False)
state_dict=torch.load("../src/models/resnet50-19c8e357.pth")
model.load_state_dict(state_dict)
model.fc = nn.Linear(2048, 80)
model.fc=nn.Flatten()

In [17]:
model=model.cuda()

In [18]:
from torch_geometric.data import Data
import tqdm
from torch_cluster import knn_graph

In [19]:
Idx=[]
Z=[]
XY=[]
Y=[]
with torch.no_grad():
    model.eval()
    for i,(imgs,xy,y,idx) in tqdm.tqdm(enumerate(trainloader),total=len(trainloader.dataset)//trainloader.batch_size):
#         print(imgs.shape,xy.shape,y.shape)
#         edge_index=knn_graph(xy,k=5)
        imgs=imgs.cuda()
        z=model(imgs).cpu()
        Z.append(z)
        XY.append(xy)
        Y.append(y)
        Idx.append(idx)
        
#         graphs.append(Data(x=z,pos=xy,edge_index=edge_index,y=y))
    #     print(graphs[i].x.shape)
#         if i==100: break
        del imgs, z, xy, y, idx

9773it [12:23, 13.15it/s]                            


In [20]:
del trainloader

In [21]:
Z=torch.cat(Z,0)
XY=torch.cat(XY,0)
Y=torch.cat(Y).flatten()
# Idx=torch.cat(Idx).flatten().numpy()

In [22]:
Idx=np.hstack(Idx).flatten()

In [48]:
Idx

array([205055,    599,    960, ..., 540211, 295242, 135772])

In [ ]:
print(f'k = {k}')

In [23]:
train_graphs=[]

for idx in tqdm.tqdm(np.unique(Idx)):
    select=(Idx==idx)
    z=Z[select]
    xy=XY[select]
    y=Y[select]
    edge_index=knn_graph(xy,k=k)
    train_graphs.append(Data(x=z,pos=xy,edge_index=edge_index,y=y))

100%|██████████| 117266/117266 [15:28<00:00, 126.25it/s]


In [26]:
val_graphs=[]
Idx=[]
Z=[]
XY=[]
Y=[]
with torch.no_grad():
    model.eval()
    for i,(imgs,xy,y,idx) in tqdm.tqdm(enumerate(valloader),total=len(valloader.dataset)//valloader.batch_size):
#         print(imgs.shape,xy.shape,y.shape)
#         edge_index=knn_graph(xy,k=5)
        imgs=imgs.cuda()
        z=model(imgs).cpu() # the 2048-dimensional embeddings provided by resnet
        Z.append(z)
        XY.append(xy)
        Y.append(y)
        Idx.append(idx)
        
#         graphs.append(Data(x=z,pos=xy,edge_index=edge_index,y=y))
    #     print(graphs[i].x.shape)
#         if i==100: break
        del imgs

413it [00:33, 12.15it/s]                         


In [27]:
Z=torch.cat(Z,0)
XY=torch.cat(XY,0)
Y=torch.cat(Y).flatten()
Idx=np.hstack(Idx).flatten()#torch.cat(Idx).flatten().numpy()

In [28]:
for idx in tqdm.tqdm(np.unique(Idx)):
    select=(Idx==idx)
    z=Z[select]
    xy=XY[select]
    y=Y[select]
    edge_index=knn_graph(xy,k=k)
    val_graphs.append(Data(x=z,pos=xy,edge_index=edge_index,y=y))

100%|██████████| 4952/4952 [00:12<00:00, 404.51it/s]


In [ ]:
del valloader

In [39]:
k

5

In [29]:

mod_select = input('COCO or resnet50 graph embeddings? (coco/resnet50) ')
# if mod_select == 'coco':
    

COCO or resnet50 graph embeddings? (coco/resnet50) resnet50


In [30]:
%%time 
### For writing new train graphs
pickle_name = f'../data/temp/{mod_select}_train_graphs_k_{k}.pkl'
ans = input(f'Are you sure you want to overwrite {pickle_name}? ([y],n)')
if ans == 'y' or ans=='':
    with open(pickle_name, 'wb') as f:
    #     val_graphs = pickle.load(f)
        pickle.dump(train_graphs, f)

Are you sure you want to overwrite ../data/temp/resnet50_train_graphs_k_3.pkl? ([y],n)y
CPU times: user 28 s, sys: 9.63 s, total: 37.6 s
Wall time: 47.1 s


In [32]:
%%time 
### For writing new val graphs
val_pickle_name = f'../data/temp/{mod_select}_val_graphs_k_{k}.pkl'

ans = input(f'Are you sure you want to overwrite {val_pickle_name}? ([y],n)')
if ans == 'y' or ans =='':
    with open(f'../data/temp/val_graphs_k_{k}.pkl', 'wb') as f:
    #     val_graphs = pickle.load(f)
        pickle.dump(val_graphs, f)

Are you sure you want to overwrite ../data/temp/resnet50_val_graphs_k_3.pkl? ([y],n)
CPU times: user 2.86 s, sys: 532 ms, total: 3.39 s
Wall time: 5.12 s


In [46]:
%%time 
### For reading train_graphs
with open(f'../data/temp/{mod_select}_train_graphs_k_{k}.pkl', 'rb') as f:
    train_graphs = pickle.load(f)
#     pickle.dump(val_graphs, f)

CPU times: user 43.4 s, sys: 15.8 s, total: 59.2 s
Wall time: 59.2 s


In [66]:
%%time 
### For reading val_graphs
with open(f'../data/temp/{mod_select}_val_graphs_k_{k}.pkl', 'rb') as f:
    val_graphs = pickle.load(f)
#     pickle.dump(val_graphs, f)

CPU times: user 1.83 s, sys: 412 ms, total: 2.24 s
Wall time: 2.23 s


In [33]:
from torch_geometric.data import DataLoader as TG_DataLoader
# https://pytorch.org/docs/stable/data.html#torch.utils.data.Subset
# Batch size should be the same as batch size for dataloader above
graph_trainloader = TG_DataLoader(train_graphs, batch_size=train_batch_size, shuffle=True)

graph_valloader=TG_DataLoader(val_graphs,batch_size=train_batch_size,shuffle=True)#[0]

In [41]:
# https://github.com/jlevy44/WSI-GTFE/blob/master/notebooks/3_fit_gnn_model.ipynb
import torch, torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv, DeepGraphInfomax, SAGEConv
from torch_geometric.nn import DenseGraphConv
from torch_geometric.utils import to_dense_batch, to_dense_adj, dense_to_sparse
from torch_geometric.nn import GINEConv
from torch_geometric.utils import dropout_adj
import torch.nn.functional as F



class GCNNet(torch.nn.Module):
    def __init__(self, inp_dim, out_dim, hidden_topology=[32,64,128,128], p=0.2, p2=0.0, drop_each=True):
        super(GCNNet, self).__init__()
        self.out_dim=out_dim
        self.convs = nn.ModuleList([GATConv(inp_dim, hidden_topology[0])]+[GATConv(hidden_topology[i],hidden_topology[i+1]) for i in range(len(hidden_topology[:-1]))])
        self.drop_edge = lambda edge_index: dropout_adj(edge_index,p=p2)[0]
        self.dropout = nn.Dropout(p)
        self.fc = nn.Linear(hidden_topology[-1], out_dim)
        self.drop_each=drop_each

    def forward(self, x, edge_index, edge_attr=None, return_attention=False):
        attention_weights=[]
        for conv in self.convs:
            if self.drop_each and self.training: edge_index=self.drop_edge(edge_index)
            x, attention = conv(x, edge_index, edge_attr,return_attention_weights=True)
            x = F.relu(x)
            attention_weights.append(attention)
        if self.training:
            x = self.dropout(x)
        x = self.fc(x)
        if return_attention: return x, attention_weights
        return x

In [42]:
model_gcn=GCNNet(2048,80, [32]*2)
model_mlp=nn.Sequential(nn.Sequential(nn.Linear(2048,32),nn.ReLU(),nn.Dropout(p=0.3)),nn.Linear(32,80))

In [43]:
# model_gcn.cuda()

In [44]:
# If there are GPUs, choose the first one for computing. Otherwise use CPU.
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)
# If 'cuda:0' is printed, it means GPU is available.

In [45]:
# model_gcn.to(device)

In [46]:
# Params Cell
k = 3
epochs = 10
fname = f'{mod_select}_graph_80_{epochs}_{k}'
batch_size=12
fname

'resnet50_graph_80_10_3'

In [47]:
### TODO: implement edge index as part of dataloader

In [48]:
loss_func = nn.CrossEntropyLoss()
opt = optim.SGD(model_gcn.parameters(), lr=0.0001, momentum=0.9)
avg_losses = []
print_freq = 100

In [49]:
for epoch in range(epochs):
    running_loss = 0.0
    for i, G in enumerate(graph_trainloader, 0):
        y=G.y#.cuda()
        z=G.x#.cuda()
        edge_index=G.edge_index#.cuda()
        
        opt.zero_grad()
        
        outputs = model_gcn(z, edge_index)
        loss = loss_func(outputs, y)
        
        loss.backward()
        
        opt.step()
        
        running_loss += loss.item()
        if i % print_freq == print_freq - 1:
            avg_loss = running_loss / print_freq
            print(f'[epoch: {epoch}, i: {i}] avg mini-batch loss: {avg_loss}')
            avg_losses.append(avg_loss)
            running_loss = 0.0
        
#         y_pred=model_gcn(z,edge_index)
#         print(y_pred.argmax(1))

[epoch: 0, i: 99] avg mini-batch loss: 4.38525438785553
[epoch: 0, i: 199] avg mini-batch loss: 4.314161739349365
[epoch: 0, i: 299] avg mini-batch loss: 4.248796699047088
[epoch: 0, i: 399] avg mini-batch loss: 4.106675815582276
[epoch: 0, i: 499] avg mini-batch loss: 3.945991053581238
[epoch: 0, i: 599] avg mini-batch loss: 3.8418880772590636
[epoch: 0, i: 699] avg mini-batch loss: 3.79966347694397
[epoch: 0, i: 799] avg mini-batch loss: 3.730191581249237
[epoch: 0, i: 899] avg mini-batch loss: 3.704592018127441
[epoch: 0, i: 999] avg mini-batch loss: 3.668990070819855
[epoch: 0, i: 1099] avg mini-batch loss: 3.5890817546844485
[epoch: 0, i: 1199] avg mini-batch loss: 3.587279965877533
[epoch: 0, i: 1299] avg mini-batch loss: 3.6070118141174317
[epoch: 0, i: 1399] avg mini-batch loss: 3.558565013408661
[epoch: 0, i: 1499] avg mini-batch loss: 3.5217103099823
[epoch: 0, i: 1599] avg mini-batch loss: 3.497951066493988
[epoch: 0, i: 1699] avg mini-batch loss: 3.4941686892509463
[epoch: 

[epoch: 1, i: 4299] avg mini-batch loss: 2.8567952144145967
[epoch: 1, i: 4399] avg mini-batch loss: 2.917203266620636
[epoch: 1, i: 4499] avg mini-batch loss: 2.844482595920563
[epoch: 1, i: 4599] avg mini-batch loss: 2.909718720912933
[epoch: 1, i: 4699] avg mini-batch loss: 2.8392962074279784
[epoch: 1, i: 4799] avg mini-batch loss: 2.858331084251404
[epoch: 1, i: 4899] avg mini-batch loss: 2.8709269630908967
[epoch: 1, i: 4999] avg mini-batch loss: 2.8746053266525267
[epoch: 1, i: 5099] avg mini-batch loss: 2.8467539048194883
[epoch: 1, i: 5199] avg mini-batch loss: 2.876728558540344
[epoch: 1, i: 5299] avg mini-batch loss: 2.823540381193161
[epoch: 1, i: 5399] avg mini-batch loss: 2.9061702346801757
[epoch: 1, i: 5499] avg mini-batch loss: 2.8779929733276366
[epoch: 1, i: 5599] avg mini-batch loss: 2.7599440383911134
[epoch: 1, i: 5699] avg mini-batch loss: 2.867840378284454
[epoch: 1, i: 5799] avg mini-batch loss: 2.8213631403446198
[epoch: 1, i: 5899] avg mini-batch loss: 2.7935

[epoch: 2, i: 8399] avg mini-batch loss: 2.5334748935699465
[epoch: 2, i: 8499] avg mini-batch loss: 2.58425847530365
[epoch: 2, i: 8599] avg mini-batch loss: 2.540086876153946
[epoch: 2, i: 8699] avg mini-batch loss: 2.4713065707683564
[epoch: 2, i: 8799] avg mini-batch loss: 2.542944362163544
[epoch: 2, i: 8899] avg mini-batch loss: 2.5188810873031615
[epoch: 2, i: 8999] avg mini-batch loss: 2.4932339918613433
[epoch: 2, i: 9099] avg mini-batch loss: 2.5763074338436125
[epoch: 2, i: 9199] avg mini-batch loss: 2.533352711200714
[epoch: 2, i: 9299] avg mini-batch loss: 2.5287701761722565
[epoch: 2, i: 9399] avg mini-batch loss: 2.568584190607071
[epoch: 2, i: 9499] avg mini-batch loss: 2.47990358710289
[epoch: 2, i: 9599] avg mini-batch loss: 2.5222666585445404
[epoch: 2, i: 9699] avg mini-batch loss: 2.5063683724403383
[epoch: 3, i: 99] avg mini-batch loss: 2.4747055733203887
[epoch: 3, i: 199] avg mini-batch loss: 2.541805433034897
[epoch: 3, i: 299] avg mini-batch loss: 2.5364820718

[epoch: 4, i: 2799] avg mini-batch loss: 2.4019240641593935
[epoch: 4, i: 2899] avg mini-batch loss: 2.337753381729126
[epoch: 4, i: 2999] avg mini-batch loss: 2.4123640203475953
[epoch: 4, i: 3099] avg mini-batch loss: 2.352957216501236
[epoch: 4, i: 3199] avg mini-batch loss: 2.3711875212192535
[epoch: 4, i: 3299] avg mini-batch loss: 2.315622411966324
[epoch: 4, i: 3399] avg mini-batch loss: 2.3803850662708284
[epoch: 4, i: 3499] avg mini-batch loss: 2.3847078812122344
[epoch: 4, i: 3599] avg mini-batch loss: 2.335813727378845
[epoch: 4, i: 3699] avg mini-batch loss: 2.4013800287246703
[epoch: 4, i: 3799] avg mini-batch loss: 2.395667736530304
[epoch: 4, i: 3899] avg mini-batch loss: 2.3599763357639314
[epoch: 4, i: 3999] avg mini-batch loss: 2.369156231880188
[epoch: 4, i: 4099] avg mini-batch loss: 2.425623118877411
[epoch: 4, i: 4199] avg mini-batch loss: 2.3645190846920014
[epoch: 4, i: 4299] avg mini-batch loss: 2.363641519546509
[epoch: 4, i: 4399] avg mini-batch loss: 2.34985

[epoch: 5, i: 6899] avg mini-batch loss: 2.286770875453949
[epoch: 5, i: 6999] avg mini-batch loss: 2.267201421260834
[epoch: 5, i: 7099] avg mini-batch loss: 2.311007753610611
[epoch: 5, i: 7199] avg mini-batch loss: 2.284876745939255
[epoch: 5, i: 7299] avg mini-batch loss: 2.283902291059494
[epoch: 5, i: 7399] avg mini-batch loss: 2.2485019254684446
[epoch: 5, i: 7499] avg mini-batch loss: 2.3147114658355714
[epoch: 5, i: 7599] avg mini-batch loss: 2.3117940390110014
[epoch: 5, i: 7699] avg mini-batch loss: 2.3127050948143006
[epoch: 5, i: 7799] avg mini-batch loss: 2.367772045135498
[epoch: 5, i: 7899] avg mini-batch loss: 2.300873830318451
[epoch: 5, i: 7999] avg mini-batch loss: 2.2144195938110354
[epoch: 5, i: 8099] avg mini-batch loss: 2.2844110321998596
[epoch: 5, i: 8199] avg mini-batch loss: 2.299226441383362
[epoch: 5, i: 8299] avg mini-batch loss: 2.2769426429271697
[epoch: 5, i: 8399] avg mini-batch loss: 2.264204573631287
[epoch: 5, i: 8499] avg mini-batch loss: 2.296391

[epoch: 7, i: 1399] avg mini-batch loss: 2.288949956893921
[epoch: 7, i: 1499] avg mini-batch loss: 2.2410192918777465
[epoch: 7, i: 1599] avg mini-batch loss: 2.195649355649948
[epoch: 7, i: 1699] avg mini-batch loss: 2.2398946940898896
[epoch: 7, i: 1799] avg mini-batch loss: 2.242698687314987
[epoch: 7, i: 1899] avg mini-batch loss: 2.219918966293335
[epoch: 7, i: 1999] avg mini-batch loss: 2.2401674926280974
[epoch: 7, i: 2099] avg mini-batch loss: 2.2149096333980562
[epoch: 7, i: 2199] avg mini-batch loss: 2.197008625268936
[epoch: 7, i: 2299] avg mini-batch loss: 2.2645156455039976
[epoch: 7, i: 2399] avg mini-batch loss: 2.2301479399204256
[epoch: 7, i: 2499] avg mini-batch loss: 2.1645080506801606
[epoch: 7, i: 2599] avg mini-batch loss: 2.2306506991386414
[epoch: 7, i: 2699] avg mini-batch loss: 2.2240951609611512
[epoch: 7, i: 2799] avg mini-batch loss: 2.197836319208145
[epoch: 7, i: 2899] avg mini-batch loss: 2.2946413588523864
[epoch: 7, i: 2999] avg mini-batch loss: 2.260

[epoch: 8, i: 5599] avg mini-batch loss: 2.162585357427597
[epoch: 8, i: 5699] avg mini-batch loss: 2.1560720944404603
[epoch: 8, i: 5799] avg mini-batch loss: 2.1812588942050932
[epoch: 8, i: 5899] avg mini-batch loss: 2.186276605129242
[epoch: 8, i: 5999] avg mini-batch loss: 2.2466735684871675
[epoch: 8, i: 6099] avg mini-batch loss: 2.1794526064395905
[epoch: 8, i: 6199] avg mini-batch loss: 2.2291969895362853
[epoch: 8, i: 6299] avg mini-batch loss: 2.187908035516739
[epoch: 8, i: 6399] avg mini-batch loss: 2.225417609214783
[epoch: 8, i: 6499] avg mini-batch loss: 2.2552047085762026
[epoch: 8, i: 6599] avg mini-batch loss: 2.2127477967739106
[epoch: 8, i: 6699] avg mini-batch loss: 2.1622272717952726
[epoch: 8, i: 6799] avg mini-batch loss: 2.2240567815303804
[epoch: 8, i: 6899] avg mini-batch loss: 2.2217354571819303
[epoch: 8, i: 6999] avg mini-batch loss: 2.1816277289390564
[epoch: 8, i: 7099] avg mini-batch loss: 2.20224023938179
[epoch: 8, i: 7199] avg mini-batch loss: 2.169

[epoch: 9, i: 9699] avg mini-batch loss: 2.1535480785369874


In [50]:
PATH = f'../src/models/{fname}.pth'
torch.save(model_gcn.state_dict(), PATH)#, _use_new_zipfile_serialization=False)
print(f'Model saved at {PATH}')

Model saved at ../src/models/resnet50_graph_80_10_3.pth


In [51]:
len(avg_losses)

970

In [52]:
pd.Series(avg_losses).to_csv(f'../data/out/avg_losses_{fname}.csv')

In [74]:
# model.state_dict()

In [44]:
# for epoch in epochs:
#     for G in graph_dataloader:
#         y=G.y.cuda()
#         z=G.x.cuda()
#         edge_index=G.edge_index.cuda()
#         y_pred=model_gcn(z,edge_index)
#         print(y_pred.argmax(1))